# MathLang Core Extended Features Demo

This notebook demonstrates the new Core Extended features in MathLang:
- **ComputationEngine**: Symbolic and numeric computation
- **ValidationEngine**: Answer validation with multiple modes
- **HintEngine**: Intelligent hint generation
- **CoreRuntime**: Orchestrated problem-solving workflow

In [ ]:
# Setup imports
import sys
sys.path.insert(0, '..')

from core.symbolic_engine import SymbolicEngine
from core.computation_engine import ComputationEngine
from core.validation_engine import ValidationEngine
from core.hint_engine import HintEngine
from core.core_runtime import CoreRuntime
from core.exercise_spec import ExerciseSpec, load_exercise_spec

## 1. ComputationEngine

The `ComputationEngine` provides symbolic manipulation and numeric evaluation.

In [ ]:
# Initialize engine
symbolic = SymbolicEngine()
engine = ComputationEngine(symbolic)

# Simplification
print("Simplification:")
print(f"  x + x + x → {engine.simplify('x + x + x')}")
print(f"  (x**2 - 1)/(x - 1) → {engine.simplify('(x**2 - 1)/(x - 1)')}")

In [ ]:
# Expansion
print("Expansion:")
print(f"  (x + 1)**2 → {engine.expand('(x + 1)**2')}")
print(f"  (x - 3)*(x + 3) → {engine.expand('(x - 3)*(x + 3)')}")

In [ ]:
# Factoring
print("Factoring:")
print(f"  x**2 - 9 → {engine.factor('x**2 - 9')}")
print(f"  x**2 + 5*x + 6 → {engine.factor('x**2 + 5*x + 6')}")

In [ ]:
# Numeric evaluation with variables
engine.bind('x', 5)
result = engine.numeric_eval('x**2 + 2*x + 1')
print(f"x**2 + 2*x + 1 (with x=5) = {result}")

## 2. ValidationEngine

The `ValidationEngine` validates student answers with different modes.

In [ ]:
validator = ValidationEngine(engine)

# Symbolic equivalence mode
spec = ExerciseSpec(
    id="ex1",
    target_expression="x**2 + 2*x + 1",
    validation_mode="symbolic_equiv"
)

result = validator.check_answer("(x + 1)**2", spec)
print(f"Is (x + 1)**2 correct? {result.is_correct}")
print(f"Message: {result.message}")

In [ ]:
# Canonical form mode - requires specific form
spec = ExerciseSpec(
    id="ex2",
    target_expression="(x - 3)*(x + 3)",
    validation_mode="canonical_form",
    canonical_form="(x - 3)*(x + 3)"
)

# This is equivalent but not in canonical form
result = validator.check_answer("x**2 - 9", spec)
print(f"\nIs x**2 - 9 in canonical form? {result.is_correct}")
print(f"Message: {result.message}")

# This is in canonical form
result = validator.check_answer("(x - 3)*(x + 3)", spec)
print(f"\nIs (x - 3)*(x + 3) in canonical form? {result.is_correct}")
print(f"Message: {result.message}")

## 3. HintEngine

The `HintEngine` generates intelligent hints based on error patterns.

In [ ]:
hint_engine = HintEngine(engine)

# Pattern-based hints
spec = ExerciseSpec(
    id="ex3",
    target_expression="x**2 + 2*x + 1",
    hint_rules={
        "x**2 + 1": "Don't forget the middle term!",
        "x**2 + x + 1": "The coefficient of x should be 2"
    }
)

hint = hint_engine.generate_hint("x**2 + 1", spec)
print(f"Student answer: x**2 + 1")
print(f"Hint: {hint.message}")
print(f"Type: {hint.hint_type}")

In [ ]:
# Heuristic hints - sign error detection
spec = ExerciseSpec(
    id="ex4",
    target_expression="x - 5"
)

hint = hint_engine.generate_hint("-x + 5", spec)
print(f"\nStudent answer: -x + 5")
print(f"Hint: {hint.message}")
print(f"Type: {hint.hint_type}")

## 4. CoreRuntime - Full Problem Lifecycle

The `CoreRuntime` orchestrates all engines for a complete problem-solving workflow.

In [ ]:
# Load exercise from file
spec = load_exercise_spec("exercises/quadratic_expansion.yaml")
print(f"Problem: {spec.metadata['description']}")
print(f"Target: {spec.target_expression}")

In [ ]:
# Create runtime
runtime = CoreRuntime(
    ComputationEngine(SymbolicEngine()),
    ValidationEngine(ComputationEngine(SymbolicEngine())),
    HintEngine(ComputationEngine(SymbolicEngine())),
    spec
)

# Simulate student attempts
attempts = [
    "x**2 + 1",           # Missing middle term
    "x**2 + x + 1",       # Wrong coefficient
    "x**2 + 2*x + 1"      # Correct
]

for i, answer in enumerate(attempts, 1):
    runtime.set("(x + 1)**2")  # Reset problem state
    result = runtime.finalize(answer)
    
    print(f"\nAttempt {i}: {answer}")
    if result["valid"]:
        print("  ✓ Correct!")
    else:
        print("  ✗ Incorrect")
        if "hint" in result["details"]:
            print(f"  💡 Hint: {result['details']['hint']['message']}")

## Try It Yourself!

Use the cells below to experiment with the engines.

In [ ]:
# Create your own exercise
my_spec = ExerciseSpec(
    id="my_exercise",
    target_expression="...",  # Your target expression
    validation_mode="symbolic_equiv",
    hint_rules={
        # Add your own hint patterns
    }
)

# Test it here
